In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import seaborn as sns
import copy

In [2]:
tf.config.experimental_run_functions_eagerly(True)
physical_devices = tf.config.experimental.list_physical_devices('GPU')
if len(physical_devices) > 0:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [3]:
df = pd.read_csv("sudoku.csv")

In [4]:
df.head()

,quizzes,solutions
0,0043002090050090010700600430060020871900074000...,8643712593258497619712658434361925871986574322...
1,0401000501070039605200080000000000170009068008...,3461792581875239645296483719658324174729168358...
2,6001203840084590720000060050002640300700800069...,6951273841384596727248369158512647392739815469...
3,4972000001004000050000160986203000403009000000...,4972583161864397252537164986293815473759641828...
4,0059103080094030600275001000300002010008200070...,4659123781894735623275681497386452919548216372...


In [5]:
number_of_samples = 250000
unsolved = df['quizzes'].iloc[:number_of_samples].values
solved = df['solutions'].iloc[:number_of_samples].values

In [6]:
X = []
X2 = []
y = []

In [7]:
for sudoku in unsolved:
    np_unsolved = np.array([int(x) for x in sudoku]).reshape(9,9,1)
    X.append(np_unsolved)
    
for sudoku in solved:
    np_solved = np.array([int(y) for y in sudoku]).reshape(81,1) -1
    y.append(np_solved)
    np_solved_x = np.array([int(x) for x in sudoku]).reshape(9,9,1)
    X2.append(np_solved_x)

In [8]:
#normal approach
np_X = np.array(X)
np_X = np_X/9
np_X -= 0.5

#second approach
np_X2 = np.array(X2)
np_X2 = np_X2/9
np_X2 -= 0.5


In [9]:
np_y = np.array(y)

In [ ]:
#del(X)
#del(y)
#del(solved)
#del(unsolved)

In [10]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [11]:
X_train, X_test, y_train, y_test = train_test_split(np_X, np_y, test_size=0.2)

In [ ]:
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Reshape, BatchNormalization, Activation, Input,Dropout
from tensorflow.keras.optimizers import Adam

In [ ]:
input_shape = (9,9,1)
batch_size = 128

In [ ]:
model = Sequential()

model.add(Conv2D(64, kernel_size=(3,3), activation='relu', padding='same', input_shape=input_shape))
model.add(BatchNormalization())
model.add(Conv2D(64, kernel_size=(3,3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(128, kernel_size=(1,1), activation='relu', padding='same'))

model.add(Flatten())
model.add(Dense(81*9))
model.add(Reshape((-1, 9)))
model.add(Activation('softmax'))

adam = Adam(learning_rate=0.001)
model.compile(loss='sparse_categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.backend import set_value

In [ ]:
early_stop = EarlyStopping(monitor='val_loss',patience=1, restore_best_weights=True)

In [ ]:
epochs =  [1, 2, 3, 4, 6, 8, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10]  # epochs for each round
digits =  [1, 2, 4, 6, 9, 12, 16, 20, 25,30, 35, 40, 45, 50, 55]  # digit to pull off

In [ ]:
i=1
for _epochs, _delete in zip(epochs, digits):
    print(f'Pass n° {i} ...')
    i += 1
    model.fit(delete_digits(X_train,_delete), y_train, 
              validation_data=(delete_digits(X_test,_delete),y_test),
              epochs=_epochs, batch_size=16, callbacks=[early_stop])
    

In [12]:
def delete_digits(np_X, digits_to_delete=1):

    np_boards = np_X.copy()
    for np_board in np_boards:
        np_board.flat[np.random.randint(0, 81, digits_to_delete)] = - 0.5  # generate blanks (replace = True)
    return np_boards

In [13]:
from tensorflow.keras.models import load_model

In [15]:
#tf.keras.models.save_model(model, "CNN2.hp5", save_format="h5")

NameError: name 'model' is not defined

In [21]:
model = tf.keras.models.load_model("CNN.hp5", compile=False)

In [14]:
model2 =  tf.keras.models.load_model("CNN2.hp5", compile=False)

In [15]:
def normalize(np_a):
    
    return (np_a/9)-0.5

In [16]:
def denormalize(np_a):
    
    return (np_a+0.5)*9

In [17]:
def solve_batch(np_boards, np_solved_boards, model):
    np_results = np.empty(np_boards.shape[0],dtype=np.bool_)
    for i in np.ndindex(np_boards.shape[:1]):
        predictions = model.predict(np_boards[i].reshape(1,9,9,1))[0]
        solved = np.argmax(predictions, axis=1).reshape((9,9)) +1
        np_results[i] = compare_sudoku(solved, (np_solved_boards[i]+1).reshape(9,9))
        print(f"Solved sudoku number: {i}")
    correct_boards = np_results.sum()
    acc = correct_boards/np_boards.shape[0]
    return correct_boards

In [18]:
def solve_human_batch(np_boards, np_solved_boards, model):
    np_results = np.empty(np_boards.shape[0],dtype=np.bool_)
    for i in np.ndindex(np_boards.shape[:1]):
        solved = solve_human_approach(np_boards[i], model)
        np_results[i] = compare_sudoku(solved, (np_solved_boards[i]+1).reshape(9,9))
        print(f"Solved sudoku number: {i}")
    correct_boards = np_results.sum()
    acc = correct_boards/np_boards.shape[0]
    return correct_boards

In [19]:
def solve_human_approach (np_sample_board, model): 
    
    #np_board = np.copy(np_sample_board)
    np_board = np.copy(np_sample_board)
    #get number of blank in sudoku board
    #blanks_count = 81 - np.count_nonzero(np_board)
    
    #for _ in range(blanks_count):
    while(1):
        np_board_4d = np_board.reshape((1,9,9,1))
        predictions = model.predict(np_board_4d)[0]
        predictions_int = np.argmax(predictions, axis=1).reshape((9,9))+1           #change range of digits from <0,8> to <1,9>
        propabilities = np.around(np.max(predictions, axis=1).reshape((9,9)), 2) 
                        
        #get position with the highest propability
        np_board = denormalize(np_board).reshape((9,9))
        zeros = (np_board == 0)
        if (zeros.sum() == 0):
            break
        blanks_indices = np.where(zeros)
        blanks_indices = np.array([(i,j) for i,j in zip(blanks_indices[0], blanks_indices[1])])
        position_to_fill = blanks_indices[propabilities[zeros].argmax()]
        #fill position with the highest propability
        value_to_fill = predictions_int[position_to_fill[0], position_to_fill[1]]
        
        np_board[position_to_fill[0], position_to_fill[1]] = value_to_fill
        np_board = normalize(np_board)
    return np_board

In [20]:
def compare_sudoku (np_sudoku1, np_sudoku2):
    #True/False array
    comparision = np_sudoku1 == np_sudoku2
    #print(np.count_nonzero(comparision))
    return comparision.all()

In [21]:
x = solve_human_batch(X_test[:1000], y_test[:1000], model2)
x

Solved sudoku number: (0,)
Solved sudoku number: (1,)
Solved sudoku number: (2,)
Solved sudoku number: (3,)
Solved sudoku number: (4,)
Solved sudoku number: (5,)
Solved sudoku number: (6,)
Solved sudoku number: (7,)
Solved sudoku number: (8,)
Solved sudoku number: (9,)
Solved sudoku number: (10,)
Solved sudoku number: (11,)
Solved sudoku number: (12,)
Solved sudoku number: (13,)
Solved sudoku number: (14,)
Solved sudoku number: (15,)
Solved sudoku number: (16,)
Solved sudoku number: (17,)
Solved sudoku number: (18,)
Solved sudoku number: (19,)
Solved sudoku number: (20,)
Solved sudoku number: (21,)
Solved sudoku number: (22,)
Solved sudoku number: (23,)
Solved sudoku number: (24,)
Solved sudoku number: (25,)
Solved sudoku number: (26,)
Solved sudoku number: (27,)
Solved sudoku number: (28,)
Solved sudoku number: (29,)
Solved sudoku number: (30,)
Solved sudoku number: (31,)
Solved sudoku number: (32,)
Solved sudoku number: (33,)
Solved sudoku number: (34,)
Solved sudoku number: (35,)
So

Solved sudoku number: (287,)
Solved sudoku number: (288,)
Solved sudoku number: (289,)
Solved sudoku number: (290,)
Solved sudoku number: (291,)
Solved sudoku number: (292,)
Solved sudoku number: (293,)
Solved sudoku number: (294,)
Solved sudoku number: (295,)
Solved sudoku number: (296,)
Solved sudoku number: (297,)
Solved sudoku number: (298,)
Solved sudoku number: (299,)
Solved sudoku number: (300,)
Solved sudoku number: (301,)
Solved sudoku number: (302,)
Solved sudoku number: (303,)
Solved sudoku number: (304,)
Solved sudoku number: (305,)
Solved sudoku number: (306,)
Solved sudoku number: (307,)
Solved sudoku number: (308,)
Solved sudoku number: (309,)
Solved sudoku number: (310,)
Solved sudoku number: (311,)
Solved sudoku number: (312,)
Solved sudoku number: (313,)
Solved sudoku number: (314,)
Solved sudoku number: (315,)
Solved sudoku number: (316,)
Solved sudoku number: (317,)
Solved sudoku number: (318,)
Solved sudoku number: (319,)
Solved sudoku number: (320,)
Solved sudoku 

Solved sudoku number: (570,)
Solved sudoku number: (571,)
Solved sudoku number: (572,)
Solved sudoku number: (573,)
Solved sudoku number: (574,)
Solved sudoku number: (575,)
Solved sudoku number: (576,)
Solved sudoku number: (577,)
Solved sudoku number: (578,)
Solved sudoku number: (579,)
Solved sudoku number: (580,)
Solved sudoku number: (581,)
Solved sudoku number: (582,)
Solved sudoku number: (583,)
Solved sudoku number: (584,)
Solved sudoku number: (585,)
Solved sudoku number: (586,)
Solved sudoku number: (587,)
Solved sudoku number: (588,)
Solved sudoku number: (589,)
Solved sudoku number: (590,)
Solved sudoku number: (591,)
Solved sudoku number: (592,)
Solved sudoku number: (593,)
Solved sudoku number: (594,)
Solved sudoku number: (595,)
Solved sudoku number: (596,)
Solved sudoku number: (597,)
Solved sudoku number: (598,)
Solved sudoku number: (599,)
Solved sudoku number: (600,)
Solved sudoku number: (601,)
Solved sudoku number: (602,)
Solved sudoku number: (603,)
Solved sudoku 

Solved sudoku number: (853,)
Solved sudoku number: (854,)
Solved sudoku number: (855,)
Solved sudoku number: (856,)
Solved sudoku number: (857,)
Solved sudoku number: (858,)
Solved sudoku number: (859,)
Solved sudoku number: (860,)
Solved sudoku number: (861,)
Solved sudoku number: (862,)
Solved sudoku number: (863,)
Solved sudoku number: (864,)
Solved sudoku number: (865,)
Solved sudoku number: (866,)
Solved sudoku number: (867,)
Solved sudoku number: (868,)
Solved sudoku number: (869,)
Solved sudoku number: (870,)
Solved sudoku number: (871,)
Solved sudoku number: (872,)
Solved sudoku number: (873,)
Solved sudoku number: (874,)
Solved sudoku number: (875,)
Solved sudoku number: (876,)
Solved sudoku number: (877,)
Solved sudoku number: (878,)
Solved sudoku number: (879,)
Solved sudoku number: (880,)
Solved sudoku number: (881,)
Solved sudoku number: (882,)
Solved sudoku number: (883,)
Solved sudoku number: (884,)
Solved sudoku number: (885,)
Solved sudoku number: (886,)
Solved sudoku 

1000

In [23]:
x = solve_batch(X_test, y_test, model2)
x

Solved sudoku number: (0,)
Solved sudoku number: (1,)
Solved sudoku number: (2,)
Solved sudoku number: (3,)
Solved sudoku number: (4,)
Solved sudoku number: (5,)
Solved sudoku number: (6,)
Solved sudoku number: (7,)
Solved sudoku number: (8,)
Solved sudoku number: (9,)
Solved sudoku number: (10,)
Solved sudoku number: (11,)
Solved sudoku number: (12,)
Solved sudoku number: (13,)
Solved sudoku number: (14,)
Solved sudoku number: (15,)
Solved sudoku number: (16,)
Solved sudoku number: (17,)
Solved sudoku number: (18,)
Solved sudoku number: (19,)
Solved sudoku number: (20,)
Solved sudoku number: (21,)
Solved sudoku number: (22,)
Solved sudoku number: (23,)
Solved sudoku number: (24,)
Solved sudoku number: (25,)
Solved sudoku number: (26,)
Solved sudoku number: (27,)
Solved sudoku number: (28,)
Solved sudoku number: (29,)
Solved sudoku number: (30,)
Solved sudoku number: (31,)
Solved sudoku number: (32,)
Solved sudoku number: (33,)
Solved sudoku number: (34,)
Solved sudoku number: (35,)
So

Solved sudoku number: (290,)
Solved sudoku number: (291,)
Solved sudoku number: (292,)
Solved sudoku number: (293,)
Solved sudoku number: (294,)
Solved sudoku number: (295,)
Solved sudoku number: (296,)
Solved sudoku number: (297,)
Solved sudoku number: (298,)
Solved sudoku number: (299,)
Solved sudoku number: (300,)
Solved sudoku number: (301,)
Solved sudoku number: (302,)
Solved sudoku number: (303,)
Solved sudoku number: (304,)
Solved sudoku number: (305,)
Solved sudoku number: (306,)
Solved sudoku number: (307,)
Solved sudoku number: (308,)
Solved sudoku number: (309,)
Solved sudoku number: (310,)
Solved sudoku number: (311,)
Solved sudoku number: (312,)
Solved sudoku number: (313,)
Solved sudoku number: (314,)
Solved sudoku number: (315,)
Solved sudoku number: (316,)
Solved sudoku number: (317,)
Solved sudoku number: (318,)
Solved sudoku number: (319,)
Solved sudoku number: (320,)
Solved sudoku number: (321,)
Solved sudoku number: (322,)
Solved sudoku number: (323,)
Solved sudoku 

Solved sudoku number: (577,)
Solved sudoku number: (578,)
Solved sudoku number: (579,)
Solved sudoku number: (580,)
Solved sudoku number: (581,)
Solved sudoku number: (582,)
Solved sudoku number: (583,)
Solved sudoku number: (584,)
Solved sudoku number: (585,)
Solved sudoku number: (586,)
Solved sudoku number: (587,)
Solved sudoku number: (588,)
Solved sudoku number: (589,)
Solved sudoku number: (590,)
Solved sudoku number: (591,)
Solved sudoku number: (592,)
Solved sudoku number: (593,)
Solved sudoku number: (594,)
Solved sudoku number: (595,)
Solved sudoku number: (596,)
Solved sudoku number: (597,)
Solved sudoku number: (598,)
Solved sudoku number: (599,)
Solved sudoku number: (600,)
Solved sudoku number: (601,)
Solved sudoku number: (602,)
Solved sudoku number: (603,)
Solved sudoku number: (604,)
Solved sudoku number: (605,)
Solved sudoku number: (606,)
Solved sudoku number: (607,)
Solved sudoku number: (608,)
Solved sudoku number: (609,)
Solved sudoku number: (610,)
Solved sudoku 

Solved sudoku number: (860,)
Solved sudoku number: (861,)
Solved sudoku number: (862,)
Solved sudoku number: (863,)
Solved sudoku number: (864,)
Solved sudoku number: (865,)
Solved sudoku number: (866,)
Solved sudoku number: (867,)
Solved sudoku number: (868,)
Solved sudoku number: (869,)
Solved sudoku number: (870,)
Solved sudoku number: (871,)
Solved sudoku number: (872,)
Solved sudoku number: (873,)
Solved sudoku number: (874,)
Solved sudoku number: (875,)
Solved sudoku number: (876,)
Solved sudoku number: (877,)
Solved sudoku number: (878,)
Solved sudoku number: (879,)
Solved sudoku number: (880,)
Solved sudoku number: (881,)
Solved sudoku number: (882,)
Solved sudoku number: (883,)
Solved sudoku number: (884,)
Solved sudoku number: (885,)
Solved sudoku number: (886,)
Solved sudoku number: (887,)
Solved sudoku number: (888,)
Solved sudoku number: (889,)
Solved sudoku number: (890,)
Solved sudoku number: (891,)
Solved sudoku number: (892,)
Solved sudoku number: (893,)
Solved sudoku 

Solved sudoku number: (1138,)
Solved sudoku number: (1139,)
Solved sudoku number: (1140,)
Solved sudoku number: (1141,)
Solved sudoku number: (1142,)
Solved sudoku number: (1143,)
Solved sudoku number: (1144,)
Solved sudoku number: (1145,)
Solved sudoku number: (1146,)
Solved sudoku number: (1147,)
Solved sudoku number: (1148,)
Solved sudoku number: (1149,)
Solved sudoku number: (1150,)
Solved sudoku number: (1151,)
Solved sudoku number: (1152,)
Solved sudoku number: (1153,)
Solved sudoku number: (1154,)
Solved sudoku number: (1155,)
Solved sudoku number: (1156,)
Solved sudoku number: (1157,)
Solved sudoku number: (1158,)
Solved sudoku number: (1159,)
Solved sudoku number: (1160,)
Solved sudoku number: (1161,)
Solved sudoku number: (1162,)
Solved sudoku number: (1163,)
Solved sudoku number: (1164,)
Solved sudoku number: (1165,)
Solved sudoku number: (1166,)
Solved sudoku number: (1167,)
Solved sudoku number: (1168,)
Solved sudoku number: (1169,)
Solved sudoku number: (1170,)
Solved sud

...

Solved sudoku number: (48972,)
Solved sudoku number: (48973,)
Solved sudoku number: (48974,)
Solved sudoku number: (48975,)
Solved sudoku number: (48976,)
Solved sudoku number: (48977,)
Solved sudoku number: (48978,)
Solved sudoku number: (48979,)
Solved sudoku number: (48980,)
Solved sudoku number: (48981,)
Solved sudoku number: (48982,)
Solved sudoku number: (48983,)
Solved sudoku number: (48984,)
Solved sudoku number: (48985,)
Solved sudoku number: (48986,)
Solved sudoku number: (48987,)
Solved sudoku number: (48988,)
Solved sudoku number: (48989,)
Solved sudoku number: (48990,)
Solved sudoku number: (48991,)
Solved sudoku number: (48992,)
Solved sudoku number: (48993,)
Solved sudoku number: (48994,)
Solved sudoku number: (48995,)
Solved sudoku number: (48996,)
Solved sudoku number: (48997,)
Solved sudoku number: (48998,)
Solved sudoku number: (48999,)
Solved sudoku number: (49000,)
Solved sudoku number: (49001,)
Solved sudoku number: (49002,)
Solved sudoku number: (49003,)
Solved s

Solved sudoku number: (49242,)
Solved sudoku number: (49243,)
Solved sudoku number: (49244,)
Solved sudoku number: (49245,)
Solved sudoku number: (49246,)
Solved sudoku number: (49247,)
Solved sudoku number: (49248,)
Solved sudoku number: (49249,)
Solved sudoku number: (49250,)
Solved sudoku number: (49251,)
Solved sudoku number: (49252,)
Solved sudoku number: (49253,)
Solved sudoku number: (49254,)
Solved sudoku number: (49255,)
Solved sudoku number: (49256,)
Solved sudoku number: (49257,)
Solved sudoku number: (49258,)
Solved sudoku number: (49259,)
Solved sudoku number: (49260,)
Solved sudoku number: (49261,)
Solved sudoku number: (49262,)
Solved sudoku number: (49263,)
Solved sudoku number: (49264,)
Solved sudoku number: (49265,)
Solved sudoku number: (49266,)
Solved sudoku number: (49267,)
Solved sudoku number: (49268,)
Solved sudoku number: (49269,)
Solved sudoku number: (49270,)
Solved sudoku number: (49271,)
Solved sudoku number: (49272,)
Solved sudoku number: (49273,)
Solved s

Solved sudoku number: (49508,)
Solved sudoku number: (49509,)
Solved sudoku number: (49510,)
Solved sudoku number: (49511,)
Solved sudoku number: (49512,)
Solved sudoku number: (49513,)
Solved sudoku number: (49514,)
Solved sudoku number: (49515,)
Solved sudoku number: (49516,)
Solved sudoku number: (49517,)
Solved sudoku number: (49518,)
Solved sudoku number: (49519,)
Solved sudoku number: (49520,)
Solved sudoku number: (49521,)
Solved sudoku number: (49522,)
Solved sudoku number: (49523,)
Solved sudoku number: (49524,)
Solved sudoku number: (49525,)
Solved sudoku number: (49526,)
Solved sudoku number: (49527,)
Solved sudoku number: (49528,)
Solved sudoku number: (49529,)
Solved sudoku number: (49530,)
Solved sudoku number: (49531,)
Solved sudoku number: (49532,)
Solved sudoku number: (49533,)
Solved sudoku number: (49534,)
Solved sudoku number: (49535,)
Solved sudoku number: (49536,)
Solved sudoku number: (49537,)
Solved sudoku number: (49538,)
Solved sudoku number: (49539,)
Solved s

Solved sudoku number: (49775,)
Solved sudoku number: (49776,)
Solved sudoku number: (49777,)
Solved sudoku number: (49778,)
Solved sudoku number: (49779,)
Solved sudoku number: (49780,)
Solved sudoku number: (49781,)
Solved sudoku number: (49782,)
Solved sudoku number: (49783,)
Solved sudoku number: (49784,)
Solved sudoku number: (49785,)
Solved sudoku number: (49786,)
Solved sudoku number: (49787,)
Solved sudoku number: (49788,)
Solved sudoku number: (49789,)
Solved sudoku number: (49790,)
Solved sudoku number: (49791,)
Solved sudoku number: (49792,)
Solved sudoku number: (49793,)
Solved sudoku number: (49794,)
Solved sudoku number: (49795,)
Solved sudoku number: (49796,)
Solved sudoku number: (49797,)
Solved sudoku number: (49798,)
Solved sudoku number: (49799,)
Solved sudoku number: (49800,)
Solved sudoku number: (49801,)
Solved sudoku number: (49802,)
Solved sudoku number: (49803,)
Solved sudoku number: (49804,)
Solved sudoku number: (49805,)
Solved sudoku number: (49806,)
Solved s

0

In [45]:
y_test[0].reshape(9,9) + 1

array([[7, 2, 6, 5, 8, 3, 9, 4, 1],
       [4, 5, 1, 7, 6, 9, 3, 2, 8],
       [8, 9, 3, 1, 4, 2, 6, 7, 5],
       [5, 1, 2, 6, 3, 8, 4, 9, 7],
       [6, 7, 4, 9, 2, 5, 1, 8, 3],
       [9, 3, 8, 4, 7, 1, 5, 6, 2],
       [1, 4, 5, 8, 9, 7, 2, 3, 6],
       [3, 6, 7, 2, 5, 4, 8, 1, 9],
       [2, 8, 9, 3, 1, 6, 7, 5, 4]])

In [ ]:
compare_sudoku(x,(y_test[150]+1).reshape(9,9))